# Import thư viện

In [1]:
from nltk.text import sent_tokenize
import os
import numpy as np
import pandas as pd
import sys
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import json
import operator
import time
import math
import string
from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter,defaultdict
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


# Import dữ liệu (gồm bộ dữ liệu Cranfield, file QUERY và file RES)

Đoc file dữ liệu cranfield

In [2]:
path = '/content/drive/MyDrive/CS419/Cranfield/Cranfield/'

In [3]:
text = []
for i in range(1,1401):
  path_file_name =path+str(i)+'.txt'
  f = open(path_file_name,'r', encoding='UTF-8')
  data = f.read()
  data = data.replace("\n", " ")
  data = data.lower()
  text.append(data)
f.close()

KeyboardInterrupt: ignored

In [ ]:
text[0]

Đọc file query

In [ ]:
path_query = '/content/drive/MyDrive/CS419/Cranfield/TEST/query.txt'
Query = []
f = open(path_query ,'r', encoding='UTF-8')
for i in range(1,226):
  data = f.readline()
  data = data.replace("\n", " ")
  data = data.lower()
  Query.append(data)
f.close()

In [ ]:
Query[0]

'1\twhat similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . '

In [ ]:
Query[1]

'2\twhat are the structural and aeroelastic problems associated with flight of high speed aircraft . '

Đọc file res

In [ ]:
path_res = '/content/drive/MyDrive/CS419/Cranfield/TEST/RES/{}.txt'
RES = list()
for index in range(0,225):
  f = open(path_res.format(index+1),'r', encoding='UTF-8')
  data = f.read()
  b = data.split()
  tempt = list()
  for i in range(1,len(b),3):
    tempt.append(int(b[i]))
  RES.append(tempt)
f.close()

In [ ]:
print(RES[0])
print(len(RES))

[184, 29, 31, 12, 51, 102, 13, 14, 15, 57, 378, 859, 185, 30, 37, 52, 142, 195, 875, 56, 66, 95, 462, 497, 858, 876, 879, 880, 486]
225


# Tiền xử lý dữ liệu

Hàm xóa các kí tự đặc biệt

In [ ]:
def remove_specific_characters(data):
  alphabet = ['q','w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m',' ']
  a = []
  b = ''
  for i in range(0,len(data)):
    if data[i] in alphabet:
      a.append(data[i])
  for i in range(0,len(a)):
    b += a[i]
  return b

In [ ]:
test = 'this is the end of the day so can i borrow your daughter a little bit *() - _ 2 3 #!#$%^&*()?>;"}:}{[]}'

In [ ]:
tw = remove_specific_characters(test)

Hàm tách từ

In [ ]:
def split_word(data):
  return word_tokenize(data)

In [ ]:
split_word(tw)

['this',
 'is',
 'the',
 'end',
 'of',
 'the',
 'day',
 'so',
 'can',
 'i',
 'borrow',
 'your',
 'daughter',
 'a',
 'little',
 'bit']

Hàm stemmer

In [ ]:
def word_stemmer(data):
  ps = nltk.stem.PorterStemmer()
  stemmed = []
  for words in data:
    stemmed.append(ps.stem(words))
  return stemmed

Hàm lemmatization

In [ ]:
def word_lemma(data):
  wordnet_lemmatizer = WordNetLemmatizer()
  lemma = []
  for words in data:
    lemma.append(wordnet_lemmatizer.lemmatize(words, pos = 'v'))
  return lemma

Hàm xóa stopwwords

In [ ]:
stop_word = stopwords.words('english')
def remove_stopwords(data):
  l = []
  for word in data:
    if word not in stop_word:
      l.append(word)
  return l

Hàm tiền xử lý có sử dụng stemmer

In [ ]:
def listwords_stem(data):
  list_words = []
  for i in range(0,len(data)):
    tempt = remove_specific_characters(data[i])
    tempt = split_word(tempt)
    tempt = word_stemmer(tempt)
    tempt = remove_stopwords(tempt)
    list_words.append(tempt)
  return list_words

Hàm tiền xử lý có sử dụng lemmatizer

In [ ]:
def listwords_lemma(data):
  list_words = []
  for i in range(0,len(data)):
    tempt = remove_specific_characters(data[i])
    tempt = split_word(tempt)
    tempt = word_lemma(tempt)
    tempt = remove_stopwords(tempt)
    list_words.append(tempt)
  return list_words

In [ ]:
t = [text[0]]

In [ ]:
listwords_lemma(t)

[['experimental',
  'investigation',
  'aerodynamics',
  'wing',
  'slipstream',
  'experimental',
  'study',
  'wing',
  'propeller',
  'slipstream',
  'make',
  'order',
  'determine',
  'spanwise',
  'distribution',
  'lift',
  'increase',
  'due',
  'slipstream',
  'different',
  'angle',
  'attack',
  'wing',
  'different',
  'free',
  'stream',
  'slipstream',
  'velocity',
  'ratios',
  'result',
  'intend',
  'part',
  'evaluation',
  'basis',
  'different',
  'theoretical',
  'treatments',
  'problem',
  'comparative',
  'span',
  'load',
  'curve',
  'together',
  'support',
  'evidence',
  'show',
  'substantial',
  'part',
  'lift',
  'increment',
  'produce',
  'slipstream',
  'due',
  'destalling',
  'boundary',
  'layer',
  'control',
  'effect',
  'integrate',
  'remain',
  'lift',
  'increment',
  'subtract',
  'destalling',
  'lift',
  'find',
  'agree',
  'well',
  'potential',
  'flow',
  'theory',
  'empirical',
  'evaluation',
  'destalling',
  'effect',
  'make',

In [ ]:
listwords_stem(t)

[['experiment',
  'investig',
  'aerodynam',
  'wing',
  'slipstream',
  'experiment',
  'studi',
  'wing',
  'propel',
  'slipstream',
  'wa',
  'made',
  'order',
  'determin',
  'spanwis',
  'distribut',
  'lift',
  'increas',
  'due',
  'slipstream',
  'differ',
  'angl',
  'attack',
  'wing',
  'differ',
  'free',
  'stream',
  'slipstream',
  'veloc',
  'ratio',
  'result',
  'intend',
  'part',
  'evalu',
  'basi',
  'differ',
  'theoret',
  'treatment',
  'thi',
  'problem',
  'compar',
  'span',
  'load',
  'curv',
  'togeth',
  'support',
  'evid',
  'show',
  'substanti',
  'part',
  'lift',
  'increment',
  'produc',
  'slipstream',
  'wa',
  'due',
  'destal',
  'boundari',
  'layer',
  'control',
  'effect',
  'integr',
  'remain',
  'lift',
  'increment',
  'subtract',
  'thi',
  'destal',
  'lift',
  'wa',
  'found',
  'agre',
  'well',
  'potenti',
  'flow',
  'theori',
  'empir',
  'evalu',
  'destal',
  'effect',
  'wa',
  'made',
  'specif',
  'configur',
  'experi'

Hàm tính số lượng term

In [ ]:
def compute_quantity_term(data):
  length = 0
  for sentence in range(0,len(data)):
    temp = remove_specific_characters(data[sentence])
    temp = split_word(temp)
    length += len(temp)
  return length

In [ ]:
compute_quantity_term(t)

139

# Lập chỉ mục

Lập chỉ mục cho từng tài liệu

In [ ]:
def create_index_for_each_doc(list_word):
  a = []
  for i in range(0,len(list_word)):
    flag = False
    for w in range(0,i):
      if list_word[i] == list_word[w]:
        flag = True
        break
    if flag == False:
      count = 1
      for j in range(i + 1,len(list_word)):
        if list_word[i] == list_word[j]:
          count += 1
      b = (list_word[i], str(count))
      a.append(b)
  return a

Lập chỉ mục cho toàn bộ toàn liệu

In [ ]:
def create_index_for_doc(list_words):
  index = []
  for i in list_words:
    index.append(create_index_for_each_doc(i))
  result = []
  for sentences in range(0,len(index)):
    for words in range(0,len(index[sentences])):
      flag = False
      sum = int(index[sentences][words][1])
      for check_sentences in range(0,sentences):
        for check_words in range(0,len(index[check_sentences])):
          if index[sentences][words][0] == index[check_sentences][check_words][0]:
            flag = True
            break
        if flag == True:
            break
      if flag == False:
        b = [sentences+1]
        count = 1
        for sentences_com in range(sentences+1,len(index)):
          for words_com in range(0,len(index[sentences_com])):
            if index[sentences][words][0] == index[sentences_com][words_com][0]:
              sum = sum + int(index[sentences_com][words_com][1])
              count += 1
              b.append(sentences_com+1)
        tempt = (index[sentences][words][0],sum,count,b)
        result.append(tempt)
  result = sorted(result)
  return result

Lập chỉ mục cho câu truy vấn (chỉ lấy term, không lấy tần số)

In [ ]:
def create_index_for_query(query_list):
  result = list()
  for i in query_list:
    result.append(create_index_for_each_doc(i))

  list_term = list()
  for index in result:
    temp = list()
    for term, tf in index:
      temp.append(term)
    list_term.append(temp)

  return list_term

Hàm trả về term, tần số, số lượng, chỉ số tài liệu trong chỉ mục

In [ ]:
def info(data):
  words = []
  frequency = []
  quantity = []
  index_doc = []
  for index in range(0,len(data)):
    words.append(data[index][0])
    frequency.append(data[index][1])
    quantity.append(data[index][2])
    index_doc.append(data[index][3])
  return words, frequency, quantity, index_doc

# Binary independence model

In [ ]:
def BIM_weights(words,numberofdoc,index_doc,quantity):
  weights = dict()
  for term in range(len(index_doc)):
    d = numberofdoc/quantity[term]
    _d = quantity[term]/numberofdoc
    for id_doc in index_doc[term]:
      if words[term] not in weights.keys():
        weights[words[term]] = {'probability':0.5,  'weight': {id_doc: math.log(0.5* d, 10)} }
      else:
        weights[words[term]]['weight'][id_doc] = math.log(0.5 * d, 10)
  return weights

In [ ]:
lw_lemma = listwords_lemma(text)
lq_lemma = listwords_lemma(Query)
rs = create_index_for_doc(lw_lemma)
words, frequency, quantity, index_doc = info(rs)

In [ ]:
rs[0]

('ab', 2, 2, [744, 924])

In [ ]:
dictionary = BIM_weights(words, 1400, index_doc, quantity)

In [ ]:
dictionary

{'ab': {'probability': 0.5,
  'weight': {744: 2.5440680443502752, 924: 2.5440680443502752}},
 'abbreviate': {'probability': 0.5, 'weight': {122: 2.8450980400142565}},
 'ability': {'probability': 0.5,
  'weight': {51: 2.3679767852945943,
   77: 2.3679767852945943,
   738: 2.3679767852945943}},
 'ablate': {'probability': 0.5,
  'weight': {553: 2.1461280356782377,
   1065: 2.1461280356782377,
   1098: 2.1461280356782377,
   1100: 2.1461280356782377,
   1241: 2.1461280356782377}},
 'ablation': {'probability': 0.5,
  'weight': {82: 1.6989700043360185,
   274: 1.6989700043360185,
   553: 1.6989700043360185,
   587: 1.6989700043360185,
   1065: 1.6989700043360185,
   1096: 1.6989700043360185,
   1097: 1.6989700043360185,
   1098: 1.6989700043360185,
   1099: 1.6989700043360185,
   1100: 1.6989700043360185,
   1101: 1.6989700043360185,
   1226: 1.6989700043360185,
   1241: 1.6989700043360185,
   1279: 1.6989700043360185}},
 'ablative': {'probability': 0.5, 'weight': {536: 2.8450980400142565}},

Tính độ liên quan giữa tài liệu và câu truy vấn

In [ ]:
def Relevant(dictionary, query_list):
  RSV = dict()                                                                     # chứa trọng số của từng tài liệu liên quan với câu truy vấn
  for term in query_list:
    if term in dictionary.keys():
      for id_doc_of_term in dictionary[term]['weight']:
        if id_doc_of_term not in RSV.keys():                                        # nếu chỉ số tài liệu (id_doc_of_term) chưa có trong RSV thì RSV[id_doc_of_term] bằng trọng số của id_doc_of_term trong dict
          RSV[id_doc_of_term] = dictionary[term]['weight'][id_doc_of_term]
        else:                                                                       # nếu chỉ số tài liệu (id_doc_of_term) đã có trong RSV thì RSV[id_doc_of_term] bằng tổng trọng số của id_doc_of_term trong dict và RSV[id_doc_of_term]
          RSV[id_doc_of_term] += dictionary[term]['weight'][id_doc_of_term]

  sorted_RSV = sorted(RSV.items(), key = operator.itemgetter(1), reverse = True)
  return sorted_RSV

Truy vấn trên từng query

In [ ]:
def query_BIM(query_list, words, quantity, index_doc, numberofdoc):
  dictionary = BIM_weights(words, numberofdoc, index_doc, quantity)
  sorted_relevant = Relevant(dictionary, query_list)
  return sorted_relevant

Truy vấn trên toàn bộ query

In [ ]:
def query_all_BIM(query_list, result, numberofdoc):
  words, frequency, quantity, index_doc = info(result)
  BIM_result = list()
  for i in query_list:
    BIM_result.append(query_BIM(i, words, quantity, index_doc, numberofdoc))
  return BIM_result, len(words)

# Tính Precision and Recall, MAP và MAPr

Hàm chuyển về dạng bool

In [ ]:
def convert_to_boolean(Query, RES):
  Query_result = list()
  for sentences in Query:
    tempt = list()
    for idx_docs, _ in sentences:
      tempt.append(idx_docs)
    Query_result.append(tempt)
  for index in range(len(Query_result)):
      for index_docs in range(len(Query_result[index])):
        if Query_result[index][index_docs] in RES[index]:
          Query_result[index][index_docs] = 1
        else:
          Query_result[index][index_docs] = 0

  return Query_result

Hàm tính chiều dài của các file trong RES (đếm số lượng tài liệu liên quan được trả về cho từng câu truy vấn trong dữ liệu mẫu)

In [ ]:
def length_of_RES(RES_read_index_docs):
  count_relevant = list()
  for i in range(len(RES_read_index_docs)):
    count_relevant.append(len(RES_read_index_docs[i]))
  return count_relevant

Hàm tính recall và precision

In [ ]:
def Recall_and_Precision(count_relevant, Query_result):
  rs = dict()
  for index in range(len(count_relevant)):
      count = 0
      recall = list()
      precision = list()
      for index_rel in range(len(Query_result[index])):
        # print(Query_result[index][index_rel])
        if Query_result[index][index_rel] == 1:
          count += 1
          recall.append(count / count_relevant[index])
          precision.append(count / (index_rel+1))
      if len(recall) == 0:
        recall.append(0)
        precision.append(0)
      rs[index+1] = {'recall': recall, 'precision': precision}

  return rs

Tính MAP, AP, AR

In [ ]:
def MAP(rs):
  AP = []
  AR = []
  for i in rs.keys():
    AP.append(np.mean(rs[i]['precision']))
    AR.append(np.mean(rs[i]['recall']))
  MAP = np.mean(AP)
  return MAP, AP, AR

Tính MAP nội suy

In [ ]:
def Recall_and_Precision_r(list_re_pre):
  rs = list()
  trec = np.arange(0,1.1,0.1)
  for i in list_re_pre.keys():
    tempt = list()
    for rec in trec:
      for index in range(len(list_re_pre[i]['recall'])):
        if rec < list_re_pre[i]['recall'][index] or rec == list_re_pre[i]['recall'][index]:
          tempt.append(max(list_re_pre[i]['precision'][index : ]))
          break
    if len(tempt) < 11:
      for i in range(len(tempt), 11):
        tempt.append(0.0)
    rs.append(tempt)
  return rs

In [ ]:
def MAPr(list_pre_APr):
  APr = list()
  for i in list_pre_APr:
    APr.append(np.mean(i))
  MAPr = np.mean(APr)
  return MAPr

In [ ]:
numberofdoc = len(text)
count_relevant = length_of_RES(RES)
query_l = create_index_for_query(lq_lemma)
query_result_test, quantity_term_test = query_all_BIM(query_l, rs, numberofdoc)

#precision, recall
Query_result_index_docs_test = convert_to_boolean(query_result_test, RES)

In [ ]:
Query_result = list()
for sentences in query_result_test:
  tempt = list()
  for idx_docs, _ in sentences:
    tempt.append(idx_docs)
  Query_result.append(tempt)

In [ ]:
print(Query_result[0])
len(Query_result[0])

[486, 329, 573, 184, 14, 12, 51, 78, 13, 332, 1361, 141, 172, 576, 1268, 746, 1194, 25, 374, 685, 572, 601, 917, 414, 944, 220, 195, 747, 792, 878, 1147, 456, 435, 216, 529, 104, 370, 875, 1066, 328, 364, 1300, 1239, 726, 584, 526, 84, 89, 908, 1144, 1113, 781, 2, 588, 663, 1003, 1313, 283, 311, 29, 665, 1202, 707, 209, 345, 416, 658, 1051, 1246, 1380, 33, 206, 235, 1325, 110, 876, 1244, 1263, 541, 817, 960, 1254, 1298, 1184, 1328, 284, 390, 1332, 1334, 34, 152, 212, 640, 1347, 373, 540, 1260, 75, 202, 729, 1064, 1163, 1170, 103, 410, 681, 704, 923, 27, 124, 1310, 163, 395, 966, 1335, 1362, 20, 28, 62, 123, 304, 349, 1213, 1236, 1336, 1375, 721, 724, 1012, 1042, 1089, 35, 157, 404, 472, 762, 964, 1028, 1065, 1232, 1304, 1327, 1365, 92, 244, 252, 280, 293, 315, 520, 675, 798, 799, 927, 991, 1155, 1180, 1218, 1255, 522, 717, 1158, 1315, 251, 253, 415, 453, 811, 914, 925, 1167, 1169, 497, 1305, 24, 36, 82, 85, 101, 142, 169, 193, 263, 302, 352, 378, 406, 466, 476, 481, 493, 552, 606, 629,

773

In [ ]:
for index in range(len(Query_result)):
    for index_docs in range(len(Query_result[index])):
      if Query_result[index][index_docs] in RES[index]:
        Query_result[index][index_docs] = 1
      else:
        Query_result[index][index_docs] = 0

In [ ]:
query_result_test

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
Query_result_index_docs_test[0]

[1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
RaP_lemma_test = Recall_and_Precision(count_relevant, Query_result_index_docs_test)
MAP_lemma_test, AP_lemma, AR_BIM_lemma = MAP(RaP_lemma_test)

In [ ]:
print(MAP_lemma_test)

0.3155194135746933


In [ ]:
RaP_lemma_test[1]

{'recall': [0.034482758620689655,
  0.06896551724137931,
  0.10344827586206896,
  0.13793103448275862,
  0.1724137931034483,
  0.20689655172413793,
  0.2413793103448276,
  0.27586206896551724,
  0.3103448275862069,
  0.3448275862068966,
  0.3793103448275862,
  0.41379310344827586,
  0.4482758620689655,
  0.4827586206896552,
  0.5172413793103449,
  0.5517241379310345,
  0.5862068965517241,
  0.6206896551724138,
  0.6551724137931034,
  0.6896551724137931,
  0.7241379310344828,
  0.7586206896551724,
  0.7931034482758621,
  0.8275862068965517,
  0.8620689655172413,
  0.896551724137931,
  0.9310344827586207],
 'precision': [1.0,
  0.5,
  0.6,
  0.6666666666666666,
  0.7142857142857143,
  0.6666666666666666,
  0.25925925925925924,
  0.21052631578947367,
  0.15,
  0.13157894736842105,
  0.06358381502890173,
  0.06666666666666667,
  0.06989247311827956,
  0.0673076923076923,
  0.06944444444444445,
  0.07373271889400922,
  0.060498220640569395,
  0.06382978723404255,
  0.06620209059233449,
  0.

In [ ]:
doc_rel = 0
for id in count_relevant:
  doc_rel += id
number_of_relevant_result = 0
for id in Query_result_index_docs_test:
  for i in id:
    if i == 1:
      number_of_relevant_result += 1

In [ ]:
doc_rel

1837

In [ ]:
number_of_relevant_result

1723

# Truy vấn

In [ ]:
lw_lemma = listwords_lemma(text)
lq_lemma = listwords_lemma(Query)

In [ ]:
lw_stem = listwords_stem(text)
lq_stem = listwords_stem(Query)

In [ ]:
def solve(query_result, quantity_of_term = "_", query_time = "_"):
  number_of_document_result = 0
  for i in query_result:
    number_of_document_result += len(i)

  Query_result_index_docs = convert_to_boolean(query_result, RES)
  number_of_relevant_result = 0
  for id in Query_result_index_docs:
    for i in id:
      if i == 1:
        number_of_relevant_result += 1

  count_relevant = length_of_RES(RES)
  list_re_pre = Recall_and_Precision(count_relevant, Query_result_index_docs)

  Map, Ap, Ar = MAP(list_re_pre)

  list_pre_APr = Recall_and_Precision_r(list_re_pre)
  Mapr = MAPr(list_pre_APr)

  doc_rel = 0
  for id in count_relevant:
    doc_rel += id

  return {'Số lượng term được sử dụng': quantity_of_term,
          'Thời gian truy vấn': query_time,
          'Tổng số tài liệu trả về': number_of_document_result,
          'Tổng số tài liệu liên quan': number_of_relevant_result,
          'Tổng số tài liệu không liên quan': number_of_document_result - number_of_relevant_result,
          'Trung bình tài liệu trả về': number_of_document_result / 225,
          'Trung bình tài liệu liên quan': number_of_relevant_result / 225,
          'Trung bình tài liệu không liên quan': (number_of_document_result - number_of_relevant_result) / 225,
          'Precision': number_of_relevant_result / number_of_document_result,
          'Recall': number_of_relevant_result / doc_rel,
          'MAP': Map,
          'MAPr': Mapr}

In [ ]:
numberofdoc = len(text)
count_relevant = length_of_RES(RES)
print(numberofdoc)
print(count_relevant[0])

1400
29


# Chạy thực nghiệm và kết quả

Dữ liệu và query sử dụng lemmatization

In [ ]:
start = time.time()
rs = create_index_for_doc(lw_lemma)                                                             # lập chỉ mục cho tập tài liệu
query_list = create_index_for_query(lq_lemma)                                                   # lập chỉ mục cho tập truy vấn
query_result_BIM_lemma, quantity_term_BIM_lemma = query_all_BIM(query_list, rs, numberofdoc)    # truy vấn
end = time.time()
time_lemma = end - start

#precision, recall
Query_result_index_docs = convert_to_boolean(query_result_BIM_lemma, RES)                 # chuyển đổi kết quả từng câu truy vấn về dạng bool
RaP_BIM_lemma = Recall_and_Precision(count_relevant, Query_result_index_docs)             # tính recall và precision
MAP_BIM_lemma, AP_BIM_lemma, AR_BIM_lemma = MAP(RaP_BIM_lemma)                            # tính MAP, AP, AR

[[(486, 6.361425499798318), (329, 5.720447442439984), (573, 4.919344068386369), (184, 4.378070557984201), (14, 4.083843747915952), (12, 3.846095587796179), (51, 3.794943065348798), (78, 3.611657710871802), (13, 3.2714214431775464), (332, 3.2714214431775464), (1361, 3.155256630876752), (141, 3.0900040566207707), (172, 3.0006428768909132), (576, 2.9633045727808587), (1268, 2.9633045727808587), (746, 2.8522558965009974), (1194, 2.743252809330688), (25, 2.6807451842298193), (374, 2.6209276883250747), (685, 2.6178180195766205), (572, 2.574815459907515), (601, 2.5341976089992513), (917, 2.528456839846763), (414, 2.5055046492109145), (944, 2.5055046492109145), (220, 2.458726489597832), (195, 2.4428656905576203), (747, 2.4428656905576203), (792, 2.4428656905576203), (878, 2.4428656905576203), (1147, 2.3661797991284086), (456, 2.363041567619082), (435, 2.3574058748209032), (216, 2.3167880239126393), (529, 2.3167880239126393), (104, 2.301029995663981), (370, 2.238102831010782), (875, 2.238102831

In [ ]:
query_result_BIM_lemma

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
time_lemma

150.5933015346527

In [ ]:
RaP_BIM_lemma

{1: {'recall': [0.034482758620689655,
   0.06896551724137931,
   0.10344827586206896,
   0.13793103448275862,
   0.1724137931034483,
   0.20689655172413793,
   0.2413793103448276,
   0.27586206896551724,
   0.3103448275862069,
   0.3448275862068966,
   0.3793103448275862,
   0.41379310344827586,
   0.4482758620689655,
   0.4827586206896552,
   0.5172413793103449,
   0.5517241379310345,
   0.5862068965517241,
   0.6206896551724138,
   0.6551724137931034,
   0.6896551724137931,
   0.7241379310344828,
   0.7586206896551724,
   0.7931034482758621,
   0.8275862068965517,
   0.8620689655172413,
   0.896551724137931,
   0.9310344827586207],
  'precision': [1.0,
   0.5,
   0.6,
   0.6666666666666666,
   0.7142857142857143,
   0.6666666666666666,
   0.25925925925925924,
   0.21052631578947367,
   0.15,
   0.13157894736842105,
   0.06358381502890173,
   0.06666666666666667,
   0.06989247311827956,
   0.0673076923076923,
   0.06944444444444445,
   0.07373271889400922,
   0.060498220640569395,
   

Dữ liệu và query sử dụng stemmer

In [ ]:
start = time.time()
rs = create_index_for_doc(lw_stem)                                                             # lập chỉ mục cho tập tài liệu
query_list_stem = create_index_for_query(lq_stem)                                                   # lập chỉ mục cho tập truy vấn
query_result_BIM_stem, quantity_term_BIM_stem = query_all_BIM(query_list_stem, rs, numberofdoc)    # truy vấn
end = time.time()
time_stem = end - start

#precision, recall
Query_result_index_docs_stem = convert_to_boolean(query_result_BIM_stem, RES)                 # chuyển đổi kết quả từng câu truy vấn về dạng bool
RaP_BIM_stem = Recall_and_Precision(count_relevant, Query_result_index_docs_stem)             # tính recall và precision
MAP_BIM_stem, AP_BIM_stem, AR_BIM_stem = MAP(RaP_BIM_stem)                                    # tính MAP, AP, AR

In [ ]:
time_stem

134.11473751068115

In [ ]:
RaP_BIM_stem

{1: {'recall': [0.034482758620689655,
   0.06896551724137931,
   0.10344827586206896,
   0.13793103448275862,
   0.1724137931034483,
   0.20689655172413793,
   0.2413793103448276,
   0.27586206896551724,
   0.3103448275862069,
   0.3448275862068966,
   0.3793103448275862,
   0.41379310344827586,
   0.4482758620689655,
   0.4827586206896552,
   0.5172413793103449,
   0.5517241379310345,
   0.5862068965517241,
   0.6206896551724138,
   0.6551724137931034,
   0.6896551724137931,
   0.7241379310344828,
   0.7586206896551724,
   0.7931034482758621,
   0.8275862068965517,
   0.8620689655172413,
   0.896551724137931,
   0.9310344827586207],
  'precision': [0.5,
   0.5,
   0.6,
   0.6666666666666666,
   0.7142857142857143,
   0.18181818181818182,
   0.16666666666666666,
   0.16666666666666666,
   0.18,
   0.136986301369863,
   0.11956521739130435,
   0.1276595744680851,
   0.10833333333333334,
   0.07650273224043716,
   0.07142857142857142,
   0.07407407407407407,
   0.07234042553191489,
   0.

In [ ]:
BIM_with_stem = solve(query_result_BIM_stem, quantity_term_BIM_stem, time_stem)
BIM_with_lemma = solve(query_result_BIM_lemma, quantity_term_BIM_lemma, time_lemma)
pd.DataFrame(
    [  BIM_with_stem,  BIM_with_lemma],
    index = [ 'BIM_With_Stem',  'BIM_with_lemma']
).astype(object).T

,BIM_With_Stem,BIM_with_lemma
Số lượng term được sử dụng,4420,5680
Thời gian truy vấn,134.114738,150.593302
Tổng số tài liệu trả về,201875,182624
Tổng số tài liệu liên quan,1748,1723
Tổng số tài liệu không liên quan,200127,180901
Trung bình tài liệu trả về,897.222222,811.662222
Trung bình tài liệu liên quan,7.768889,7.657778
Trung bình tài liệu không liên quan,889.453333,804.004444
Precision,0.008659,0.009435
Recall,0.951551,0.937942
